# component.base_component

> TODO fill in description

In [ ]:
#| default_exp comp.base_component

In [ ]:
#| hide
from nbdev.showdoc import *;

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
from __future__ import annotations
import asyncio
from abc import ABC, abstractmethod
import typing
from typing import Type, Optional, Callable, Any, Union, Tuple, Coroutine, List
from enum import Enum
import uuid
from datetime import datetime, timezone
from inspect import signature

import fbdev
from fbdev.utils import AttrContainer, TaskManager, AddressableMixin, SingletonMeta, StateCollection, StateHandler
from fbdev.comp.packet import Packet
from fbdev.comp.port import PortType, PortSpec, PortSpecCollection, Port, PortCollection

In [ ]:
#|hide
show_doc(fbdev.comp.BaseComponent)

---

### BaseComponent

>      BaseComponent ()

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [ ]:
#|export
class BaseComponent(ABC):
    is_factory = False
    
    port_specs = PortSpecCollection(
        PortSpec(PortType.MESSAGE, 'started'),
        PortSpec(PortType.MESSAGE, 'terminated')
    )
    
    def __init_subclass__(cls, inherit_ports=True, **kwargs):
        if inherit_ports and 'port_specs' in cls.__dict__:
            cls.port_specs.update(cls.__bases__[0].port_specs)
        cls.port_specs.make_readonly()
        
        if not cls.is_factory and len(signature(cls.__init__).parameters) > 1:
            raise RuntimeError(f"Invalid signature in {cls.__name__}.__init__. No arguments are allowed after `self` in components, unless it is a component factory. Got {str(signature(cls.__init__))}.")
        
    def __init__(self):
        self._task_manager = TaskManager(self)
        self._ports = PortCollection(self.port_specs)
        self._config = AttrContainer({}, obj_name="Component.config")
        self.__terminated = False
        self.__base_constructor_was_called = True
         
    @property
    def ports(self) -> PortCollection: return self._ports
    @property
    def config(self) -> AttrContainer: return self._config
    @property
    def states(self) -> StateCollection: return self._states

    def __check_base_constructor_was_called(self):
        try: return self.__base_constructor_was_called
        except AttributeError: return False

    async def start(self):
        if not self.__check_base_constructor_was_called():
            raise RuntimeError(f"{BaseComponent.__name__}.__init__() was not called in component {self.__class__.__name__}.")
        if self.__terminated:
            raise RuntimeError(f"Component {self.__class__.__name__} is terminated.")
        if self.is_factory:
            raise RuntimeError(f"Component {self.__class__.__name__} is a component factory.")
        await self._send_message('started')
        await self._update_config()
        await self._initialise()
    
    @abstractmethod
    async def _initialise(self):
        raise NotImplementedError()
    
    async def terminate(self):
        if self.__terminated: raise RuntimeError(f"Component {self.__class__.__name__} is already terminated.")
        await self._task_manager.destroy()
        self.__terminated = True
        await self._send_message('terminated')
    
    async def _update_config(self):
        async def _set_config_task(port):
            packet = await port.get()
            data = await packet.consume()
            self.set_config(port.name, data)
        tasks = []
        for port_name, port in self.ports.config.items():
            if self.port_specs.config[port_name].is_optional:
                if port.put_awaiting.get():
                    await _set_config_task(port)
            else:
                tasks.append(asyncio.create_task(_set_config_task(port)))
        await asyncio.gather(*tasks)
    
    def _set_config(self, name:str, value:Any):
        if name not in self.port_specs.config.keys():
            raise ValueError(f"Config port {name} is not a valid config port for component {self.__class__.__name__}.")
        self.config._set(name, value)
        
    async def _await_signal(self, name:str):
        packet = await self.ports.signal[name].get()
        await packet.consume()
        
    async def _send_message(self, name:str):
        if self.ports.message[name].states.get_awaiting.get():
            await self.ports.message[name].put(Packet.get_empty())
    
    async def put_packet(self, port_name:str, packet:Packet):
        if not self.ports[port_name].is_input_port:
            raise ValueError(f"Port {port_name} is not an input port.")
        await self.ports[port_name]._put(packet)
    
    async def get_packet(self, port_name:str) -> Packet:
        if self.ports[port_name].is_input_port:
            raise ValueError(f"Port {port_name} is not an output port.")
        return await self.ports[port_name]._get()
    
    @classmethod
    def _create_component_class(cls, component_name=None, class_attrs={}, init_args=[], init_kwargs={}) -> Type[BaseComponent]:
        if not cls.is_factory:
            raise ValueError(f"{cls.__name__} is not a component factory.")
        if component_name is None:
            if cls.__name__.endswith("Factory"):
                component_name = cls.__name__[:-len("Factory")]
            else:
                component_name = cls.__name__
        return type(component_name, (cls,), {
            '__init__': lambda self: cls.__init__(self, *init_args, **init_kwargs),
            'parent_factory' : cls,
            **class_attrs,
            'is_factory' : False,
        })

    @classmethod
    def create_component(cls) -> Type[BaseComponent]:
        """Creates a new instance of the component class, with the given arguments.
        Overload to modify the behaviour (for example, to allow modification of `port_spec`)
        """
        raise NotImplementedError()

In [ ]:
class FooComponent(BaseComponent):
    async def _initialise(self): pass
    
comp_process = FooComponent()
    
async def check_started():
    await comp_process.get_packet('message.started')
    print("Component process started")
    
async def check_terminated():
    await comp_process.get_packet('message.terminated')
    print("Component process terminated")

await asyncio.gather(
    asyncio.create_task(check_started()),
    asyncio.create_task(check_terminated()),
    asyncio.create_task(comp_process.start()),
    asyncio.create_task(comp_process.terminate()),
);

Component process started
Component process terminated


In [ ]:
class MyComponentFactory(BaseComponent):    
    is_factory = True

    @classmethod
    def create_component(cls, my_attr) -> Type[BaseComponent]:
        return cls._create_component_class(class_attrs={
            'my_attr' : my_attr
        })
    
    async def _initialise(self):
        print(self.my_attr)
        
comp = MyComponentFactory.create_component('hello world')
print('Component:', comp.__name__)
comp_process = comp()

await asyncio.gather(
    asyncio.create_task(comp_process.start()),
    asyncio.create_task(comp_process.terminate())
);

Component: MyComponent
hello world
